In [1]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LayerNormalization,MaxPool2D
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.utils import to_categorical 
from python_speech_features import mfcc, logfbank
import csv
import os
from keras import layers
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,

In [ ]:
header = 'filename'
for i in range(1, 26):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'cry no_cry'.split()
for g in genres:
    for filename in os.listdir(f'ba_cry/{g}'):
        songname = f'ba_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=7)
        mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=25)
        to_append = f'{filename}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [10]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
# X =np.array(data.iloc[:, :-1], dtype = float)
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = Sequential()
# model.add(Conv2D(16, (3, 3), activation='relu', strides=(1, 1),padding='same', input_shape=X_train.shape[1]))
# model.add(Conv2D(32, (3, 3), activation='relu', strides=(1, 1),padding='same')) 
# model.add(Conv2D(64, (3, 3), activation='relu', strides=(1, 1),padding='same'))
# model.add(MaxPool2D ((2, 2)))
# model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,shuffle=True,epochs=50,batch_size=128)

In [2]:
from keras.models import load_model
model = load_model('ff_nn_v3_Ac88_best.h5')
model.load_weights('ff_nn_v3_Ac88_best_weights.h5')

In [3]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [4]:
header = 'filename'
for i in range(1, 26):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [5]:
file = open('predict.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
y, sr = librosa.load('output.wav', mono=True, duration=5)
mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=25)
to_append = f'output.wav '   
for e in mfcc:
    to_append += f' {np.mean(e)}'
file = open('predict.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())
live_data = pd.read_csv('predict.csv')
live_data = live_data.drop(['filename'],axis=1)


In [6]:
op=list(to_append.split())
print(op.pop(0))

output.wav


In [11]:

print(op)
Xnew = scaler.transform([op])
print(Xnew)
ynew = model.predict_classes(Xnew)
print("Predicted=%s" % (ynew))
if ynew==0:
    print('Cry')
else:
    print('No cry')

['-383.49456787109375', '100.76104736328125', '13.4022216796875', '32.64509963989258', '12.863853454589844', '4.169948101043701', '2.1110963821411133', '-1.8287122249603271', '-10.638762474060059', '10.564427375793457', '0.8306987881660461', '4.8464837074279785', '1.9661191701889038', '3.62606143951416', '3.008317708969116', '6.592686176300049', '-4.3055419921875', '1.432627558708191', '-0.5799728035926819', '3.029869556427002', '-3.412336587905884', '0.638687789440155', '-1.7327020168304443', '2.0981388092041016', '-4.872173309326172']
[[-0.4745465   1.04202753  1.27148003  1.59036564  1.38051076  0.10011212
   0.62507151 -0.5104022  -1.0130688   1.04684654  0.27183806  0.40394291
   0.40240906  0.01695611  0.40928814  0.39679809 -0.71165938 -0.19917914
   0.11264307  0.13188081 -0.40059197 -0.37477491  0.01371644  0.16830556
  -1.03931743]]
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. i

In [ ]:
# Xnew = np.array([live_data.iloc[0][0:25]])
# print(Xnew)
# ynew = model.predict_classes(Xnew)
# print("Predicted=%s" % (ynew))
# if ynew==0:
#     print('Cry')
# else:
#     print('No cry')

In [15]:
import pyaudio
import wave
import pathlib
import threading
import csv
import time

# t1 = threading.Thread(target=print_square, args=(10,))
# t2 = threading.Thread(target=print_cube, args=(10,))

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

stop = 0


p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")


def predict(name):
    #Doing prediction
    cmap = plt.get_cmap('inferno')
    plt.figure(figsize=(8,8))
    pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
    # for filename in os.listdir(f'b_cry/{g}'):
    songname = name+'.wav'
    y, sr = librosa.load(songname, mono=True, duration=7)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig(f'live_data/{name}.png')
    plt.clf()

    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()

    file = open(f'{name}.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
    # genres = 'discomfort belly_pain burping hungry'.split()
    # for g in genres:
    #     for filename in os.listdir(f'b_cry/{g}'):
    # songname = f'b_cry/{g}/{filename}'
#     songname = f'output.wav'
    y, sr = librosa.load(songname, mono=True, duration=5)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=25)
    to_append = f'{name}.wav '   
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    # to_append += f' {model.predict(live_data.iloc[0])}'
    #to_append += f' live_data_genre'
    file = open(f'{name}.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())
    live_data = pd.read_csv(f'{name}.csv')
    live_data = live_data.drop(['filename'],axis=1)

    #print(live_data.iloc[0][0:26])
    op=list(to_append.split())
    op.pop(0)
#     print(Xnew)
    Xnew = scaler.transform([op])
    ynew = model.predict_classes(Xnew)

    print("Predicted=%s" % (ynew))
    if ynew==0:
        print('Cry')
    else:
        print('No cry')


# def record_audio():
for j in range(0,4):
    frames = []
    songname = "output"+str(j)
    WAVE_OUTPUT_FILENAME = "output"+str(j)+".wav"
    print(WAVE_OUTPUT_FILENAME)
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    
    print("Predicting")
#     for i in range(0,5):
#         print(str(i*100))
#         time.sleep(0.5)
    t2 = threading.Thread(target=predict, args=(songname,))
    t2.start()
    
# if __name__ == "__main__":
    # creating thread
# t1 = threading.Thread(target=record_audio, args=())
# t2 = threading.Thread(target=predict, args=())

# while(stop==0):
# t1.start()
# t2.start()
# t1.join()
# t2.join()
# both threads completely executed
print("Done!")

* recording
output0.wav
Predicting
output1.wav
Predicted=[0]
Cry
Predicting
output2.wav
Predicted=[0]
Cry
Predicting
output3.wav
Predicted=[0]
Cry
Predicting
Done!


<Figure size 432x288 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

Predicted=[0]
Cry


In [ ]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

In [ ]:
# from keras.models import load_model
# model.save("ff_nn_v_Ac88.h5")

In [ ]:
# model.save_weights('ff_nn_v_Ac88_weights.h5')